<a href="https://colab.research.google.com/github/firstiaaa/Pemb_Mesin/blob/main/UTS_ML_Kelompok_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UTS Machine Learning Kelompok 3 TI - 3H
  - Brilyan Satria Wahyuda | 2241720019
  - Firstia Aulia Wida Azizah | 2241720135
  - Fransiscus Farrel Edric Wijanarko | 2241720032
  - M. Tryo Bagus Anugerah Putra | 2241720053  

# Intro

Pada kuis ini, Anda diminta untuk melakukan klasifikasi citra wajah dengan menggunakan dataset [CISIA Webface](https://www.kaggle.com/datasets/ntl0601/casia-webface). Perhatian! Dataset ini cukup besar (2.8Gb) dari 500.000 gambar dan 10.575 subjek (label)
Spesifikasi pengerjaan UTS yang harus dipehuni adalah,

1. Pelajari tentang CISIA Webface!
2. Dikarenakan data gambar dari CISIA masih dalam 1 direktori besar, Anda perlu melakukan proses split antara data latih dan data uji. Anda dapat melakukan ini secara manual (langsung dari direktori) atau secara logikal dengan listing direktori. (10 poin)
2. Lakukan proses pra pengolahan data. Anda wajib dapat menjelaskan proses pra pengolahan data yang dilakukan. (20 poin)
3. Lakukan proses ekstraksi fitur. Fitur yang digunakan bebas. Anda wajib dapat menjelasakan fitur yang digunakan (30 poin)
4. Buat model NN dengan arsitektur yang kelompok Anda rancang sendiri. Model arsitektur bebas (jumlah layer, jumlah node, fungsi aktiviasi). (30 poin)
6. Evaluasi performa model NN kelompok Anda dengan metrik akurasi, *precision*, *recall*, dan *F1-Score*. Jelaskan maksud dari metric-metric tersebut!

# Boilerplate

Berikut merupakan boilerplate code yang dapat Anda gunakan sebagai acuan dasar pengerjaan kuis.
Anda diperkenankan untuk **menambah** ataupun **mengurangi** bagian boilerplate yang disediakan.

---

# Preparation

In [ ]:
# Load required library
# Import Required Library
import kagglehub
import os
import random
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from collections import Counter
from PIL import Image, UnidentifiedImageError
from skimage.color import rgb2gray

# Load Data

In [ ]:
'''
    Bagian ini dapat Anda gunakan untuk melakukan proses loading data
    dan juga proses split antara data latih dan data uji berdasarkan direktori gambar
'''

# Unduh dataset CASIA-Webface ke folder tujuan
path = kagglehub.dataset_download("ntl0601/casia-webface")
destination_path = "/dataset/casia-webface/"

# Cek apakah direktori tujuan ada, jika tidak buat
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# Pindahkan semua file dari folder download ke folder tujuan
if path and os.path.exists(path):
    for filename in os.listdir(path):
        shutil.move(os.path.join(path, filename), os.path.join(destination_path, filename))
    print("Dataset files moved to:", destination_path)
else:
    print("Failed to download dataset or dataset path does not exist.")

Dataset files moved to: /dataset/casia-webface/


In [ ]:
# Fungsi untuk melakukan sampling data
def sample_images(data_dir, min_images_per_person=100, max_images_per_person=100, max_person=50):
    images = []
    labels = []
    class_names = []
    person_count = 0  # Menghitung jumlah orang yang diproses

    # Iterasi semua folder dan file dalam dataset
    for root, dirs, files in os.walk(data_dir):
        if len(files) >= min_images_per_person and person_count < max_person:
            class_name = os.path.basename(root)

            # Pastikan hanya menambahkan kelas baru jika belum ada
            if class_name not in class_names:
                class_names.append(class_name)
                person_count += 1

            label = class_names.index(class_name)
            print(f"Loading images from {root}...")  # Debugging log

            # Batas jumlah gambar yang diproses per orang
            image_count = 0
            for img_name in files:
                if image_count >= max_images_per_person:
                    break  # Hentikan jika mencapai batas gambar per orang

                img_path = os.path.join(root, img_name)

                if os.path.isfile(img_path):
                    try:
                        with Image.open(img_path) as img:
                            img = img.convert('RGB').resize((64, 64))
                            img = np.array(img) / 255.0

                            if img.shape == (64, 64, 3):
                                images.append(img)
                                labels.append(label)
                                image_count += 1  # Tambah jumlah gambar yang valid
                            else:
                                print(f"Skipping {img_name}: Invalid shape {img.shape}")
                    except (UnidentifiedImageError, OSError) as e:
                        print(f"Skipping {img_name}: {e}")

    return np.array(images), np.array(labels), class_names

# Set Path dan Sampling
DATASET_PATH = '/dataset/casia-webface/casia-webface'

# Ambil sampel data untuk 50 subjek dengan masing-masing 100 gambar
images, labels, class_names = sample_images(DATASET_PATH)

print("labels:", labels)

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, stratify=labels)
print(test_labels.shape)

Loading images from /dataset/casia-webface/casia-webface/009039...
Loading images from /dataset/casia-webface/casia-webface/009544...
Loading images from /dataset/casia-webface/casia-webface/008339...
Loading images from /dataset/casia-webface/casia-webface/000953...
Loading images from /dataset/casia-webface/casia-webface/000105...
Loading images from /dataset/casia-webface/casia-webface/002966...
Loading images from /dataset/casia-webface/casia-webface/005795...
Loading images from /dataset/casia-webface/casia-webface/004088...
Loading images from /dataset/casia-webface/casia-webface/000955...
Loading images from /dataset/casia-webface/casia-webface/000136...
Loading images from /dataset/casia-webface/casia-webface/001002...
Loading images from /dataset/casia-webface/casia-webface/001313...
Loading images from /dataset/casia-webface/casia-webface/000157...
Loading images from /dataset/casia-webface/casia-webface/008700...
Loading images from /dataset/casia-webface/casia-webface/00460

# Preprocessing

In [ ]:
'''
    Pada bagian ini Anda diperbolehkan untuk melakukan proses pra pengolahan data (preprocessing) sesuai dengan kebutuhan. Pra pengolahan data dapat berupa,

    1. Standardisasi nilai fitur ataupun label
    2. Penyesuaian ukuran gambar
    3. Perubahan colorspace gambar
    4. dsb
'''

# Preprocessing kita terdapat pada saat load data, yakni resize ukurang gambar menjadi 64X64

'\n    Pada bagian ini Anda diperbolehkan untuk melakukan proses pra pengolahan data (preprocessing) sesuai dengan kebutuhan. Pra pengolahan data dapat berupa,\n\n    1. Standardisasi nilai fitur ataupun label\n    2. Penyesuaian ukuran gambar\n    3. Perubahan colorspace gambar\n    4. dsb\n'

# Menggunakan Model ANN tanpa fitur

# Features Extraction

In [ ]:
'''
    Anda dapat melakukan proses ekstrasi fitur apapun sesuai dengan yang apa yang Anda inginkan
'''

'\n    Anda dapat melakukan proses ekstrasi fitur apapun sesuai dengan yang apa yang Anda inginkan\n'

# Build Model

In [ ]:
'''
    Pada bagian ini lakukan proses pembuatan model NN,
'''

'\n    Pada bagian ini lakukan proses pembuatan model NN,\n'

In [ ]:
# Definisikan path tempat menyimpan model
checkpoint_path = "model_checkpoint_best.keras"

# Buat callback ModelCheckpoint
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,      # Path untuk menyimpan model
    monitor='val_accuracy',        # Metrik yang dipantau, misalnya 'val_accuracy' atau 'val_loss'
    save_best_only=True,           # Simpan hanya jika model lebih baik dari sebelumnya
    save_weights_only=False,       # Simpan model secara lengkap (termasuk arsitektur dan bobot)
    mode='max',                    # Mode 'max' karena kita ingin akurasi tertinggi
    verbose=1                      # Aktifkan untuk melihat pesan ketika model disimpan
)

LABELS_LIMIT = len(np.unique(train_labels))

# Arsitektur ANN
model = Sequential()
model.add(Flatten(input_shape=(64, 64, 3)))  # Ubah ukuran gambar ke (64, 64) sesuai dengan memori Colab
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(LABELS_LIMIT, activation='softmax'))


# Kompilasi Model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Training Model
history = model.fit(train_images, train_labels, epochs=100, validation_data=(test_images, test_labels))

Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.0250 - loss: 4.0986 - val_accuracy: 0.0450 - val_loss: 3.8547
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.0471 - loss: 3.8433 - val_accuracy: 0.0440 - val_loss: 3.7503
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.0505 - loss: 3.7529 - val_accuracy: 0.0510 - val_loss: 3.6520
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.0762 - loss: 3.6070 - val_accuracy: 0.0820 - val_loss: 3.5098
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.1012 - loss: 3.4642 - val_accuracy: 0.0930 - val_loss: 3.3741
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.1161 - loss: 3.3127 - val_accuracy: 0.1600 - val_loss: 3.1530
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.1703 - loss: 3.1072 - val_accuracy: 0.1490 - val_loss: 3.1109
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.2039 - loss: 2.9662 - 

# Evaluation

In [ ]:
'''
    Pada bagian ini, lakukan evaluasi terhadap data training dan data testing
'''

'\n    Pada bagian ini, lakukan evaluasi terhadap data training dan data testing\n'

In [ ]:
# Evaluasi Model
# Prediksi pada data uji
y_pred = model.predict(test_images)
y_pred_labels = np.argmax(y_pred, axis=1)  # Konversi probabilitas ke label

# Hasil evaluasi menggunakan classification report
report = classification_report(test_labels, y_pred_labels, target_names=[f'Class {i}' for i in range(LABELS_LIMIT)])

print("Classification Report:")
print(report)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.20      0.35      0.25        20
     Class 1       0.48      0.50      0.49        20
     Class 2       0.80      0.40      0.53        20
     Class 3       0.29      0.70      0.41        20
     Class 4       0.50      0.70      0.58        20
     Class 5       0.54      0.35      0.42        20
     Class 6       0.55      0.55      0.55        20
     Class 7       0.46      0.60      0.52        20
     Class 8       0.33      0.45      0.38        20
     Class 9       0.29      0.30      0.29        20
    Class 10       0.30      0.35      0.33        20
    Class 11       0.93      0.70      0.80        20
    Class 12       0.37      0.55      0.44        20
    Class 13       0.42      0.50      0.45        20
    Class 14       0.39      0.45      0.42        20
    Class 15       0.73      0.55      0.63        20
    Class 16       

# Menggunakan Model ANN dengan fitur HOG Grayscale dan PCA

## Features Extraction

In [ ]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Fungsi untuk melakukan ekstraksi fitur HOG
def extract_hog_features(images):
    hog_features = []
    for img in images:
        # Konversi gambar ke grayscale
        gray_img = rgb2gray(img)
        # Ekstraksi fitur HOG
        features = hog(gray_img, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                       block_norm='L2-Hys', visualize=False, feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

# Ekstraksi fitur HOG dari data latih dan uji
train_hog_features = extract_hog_features(train_images)
test_hog_features = extract_hog_features(test_images)

# Standarisasi fitur sebelum PCA
scaler = StandardScaler()
train_hog_features = scaler.fit_transform(train_hog_features)
test_hog_features = scaler.transform(test_hog_features)

# Terapkan PCA untuk mengurangi dimensi fitur HOG
pca = PCA(n_components=200)  # Atur jumlah komponen sesuai kebutuhan
train_pca_features = pca.fit_transform(train_hog_features)
test_pca_features = pca.transform(test_hog_features)

print("Train PCA Features Shape:", train_pca_features.shape)
print("Test PCA Features Shape:", test_pca_features.shape)


Train PCA Features Shape: (4000, 200)
Test PCA Features Shape: (1000, 200)


## Build Model

In [ ]:
# Hitung jumlah komponen PCA untuk dimensi input
input_dim = train_pca_features.shape[1]

print(input_dim)

# Bangun model ANN yang sesuai dengan input PCA
model = Sequential()
model.add(Input(shape=(input_dim,)))  # Sesuaikan input dengan dimensi hasil PCA
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(LABELS_LIMIT, activation='softmax'))


# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Melatih model dengan data PCA
history = model.fit(
    train_pca_features,           # Fitur latih PCA
    train_labels,                 # Label latih
    epochs=20,
    callbacks=[checkpoint_callback],  # Callback untuk penyimpanan model terbaik
    validation_data=(test_pca_features, test_labels)
)



200
Epoch 1/20
120/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0813 - loss: 4.0547
Epoch 1: val_accuracy improved from -inf to 0.31800, saving model to model_checkpoint_best.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.0850 - loss: 4.0317 - val_accuracy: 0.3180 - val_loss: 2.8018
Epoch 2/20
117/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5295 - loss: 1.9657
Epoch 2: val_accuracy improved from 0.31800 to 0.50700, saving model to model_checkpoint_best.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5310 - loss: 1.9552 - val_accuracy: 0.5070 - val_loss: 2.0320
Epoch 3/20
111/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7699 - loss: 0.9805
Epoch 3: val_accuracy improved from 0.50700 to 0.55900, saving model to model_checkpoint_best.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7690 - loss: 0.9784 - val_accuracy: 0.5590 - val_loss: 1.8286
Epoch 4/20
119/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8947 - loss: 0.5161
Epoc

## Evaluasi

In [ ]:
# Prediksi pada data uji
y_pred = model.predict(test_pca_features)
y_pred_labels = np.argmax(y_pred, axis=1)  # Konversi probabilitas ke label

# Hasil evaluasi menggunakan classification report
report = classification_report(test_labels, y_pred_labels, target_names=[f'Class {i}' for i in range(LABELS_LIMIT)])

print("Classification Report:")
print(report)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.39      0.45      0.42        20
     Class 1       0.70      0.70      0.70        20
     Class 2       0.67      0.70      0.68        20
     Class 3       0.48      0.50      0.49        20
     Class 4       0.54      0.70      0.61        20
     Class 5       0.68      0.75      0.71        20
     Class 6       0.73      0.55      0.63        20
     Class 7       0.57      0.65      0.60        20
     Class 8       0.50      0.60      0.55        20
     Class 9       0.45      0.50      0.48        20
    Class 10       0.41      0.55      0.47        20
    Class 11       0.86      0.95      0.90        20
    Class 12       0.72      0.65      0.68        20
    Class 13       0.55      0.60      0.57        20
    Class 14       0.46      0.30      0.36        20
    Class 15       0.60      0.75      0.67        20
    Class 16       

# Menggunakan Model ANN dengan fitur HOG RGB dan PCA

## Features Extraction

In [ ]:
# Fungsi untuk melakukan ekstraksi fitur HOG pada RGB
def extract_hog_features_rgb(images):
    hog_features = []
    for img in images:
        # Ekstraksi HOG untuk setiap channel warna (R, G, dan B)
        hog_r = hog(img[:, :, 0], pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                    block_norm='L2-Hys', visualize=False, feature_vector=True)
        hog_g = hog(img[:, :, 1], pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                    block_norm='L2-Hys', visualize=False, feature_vector=True)
        hog_b = hog(img[:, :, 2], pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                    block_norm='L2-Hys', visualize=False, feature_vector=True)

        # Gabungkan fitur HOG dari ketiga channel
        features = np.hstack((hog_r, hog_g, hog_b))
        hog_features.append(features)

    return np.array(hog_features)

# Ekstraksi fitur HOG RGB dari data latih dan uji
train_hog_features_rgb = extract_hog_features_rgb(train_images)
test_hog_features_rgb = extract_hog_features_rgb(test_images)

# Standarisasi fitur sebelum PCA
train_hog_features_rgb = scaler.fit_transform(train_hog_features_rgb)
test_hog_features_rgb = scaler.transform(test_hog_features_rgb)

# Terapkan PCA untuk mengurangi dimensi fitur HOG
pca_rgb = PCA(n_components=400)  # Atur jumlah komponen sesuai kebutuhan
train_pca_features_rgb = pca_rgb.fit_transform(train_hog_features_rgb)
test_pca_features_rgb = pca_rgb.transform(test_hog_features_rgb)

print("Train PCA Features Shape:", train_pca_features_rgb.shape)
print("Test PCA Features Shape:", test_pca_features_rgb.shape)

Train PCA Features Shape: (4000, 400)
Test PCA Features Shape: (1000, 400)


## Build Model

In [ ]:
# Hitung jumlah komponen PCA untuk dimensi input
input_dim = train_pca_features_rgb.shape[1]

print(input_dim)

# Bangun model ANN yang sesuai dengan input PCA
model = Sequential()
model.add(Input(shape=(input_dim,)))  # Sesuaikan input dengan dimensi hasil PCA
model.add(Dense(400, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(LABELS_LIMIT, activation='softmax'))

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Melatih model dengan data PCA
history = model.fit(
    train_pca_features_rgb,           # Fitur latih PCA
    train_labels,                 # Label latih
    epochs=10,
    batch_size=32,
    callbacks=[checkpoint_callback],  # Callback untuk penyimpanan model terbaik
    validation_data=(test_pca_features_rgb, test_labels)
)

400
Epoch 1/10
122/125 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1546 - loss: 4.0357
Epoch 1: val_accuracy did not improve from 0.60700
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.1589 - loss: 4.0063 - val_accuracy: 0.4740 - val_loss: 2.0804
Epoch 2/10
118/125 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8305 - loss: 0.7529
Epoch 2: val_accuracy did not improve from 0.60700
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8298 - loss: 0.7520 - val_accuracy: 0.5560 - val_loss: 1.7629
Epoch 3/10
117/125 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9842 - loss: 0.1382
Epoch 3: val_accuracy did not improve from 0.60700
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9844 - loss: 0.1369 - val_accuracy: 0.6010 - val_loss: 1.6925
Epoch 4/10
118/125 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0241
Epoch 4: val_accuracy improved from 0.60700 to 0.61800, saving model to model_checkpoint_best.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accu

## Evaluasi

In [ ]:
# Prediksi pada data uji
y_pred = model.predict(test_pca_features_rgb)
y_pred_labels = np.argmax(y_pred, axis=1)  # Konversi probabilitas ke label

# Hasil evaluasi menggunakan classification report
report = classification_report(test_labels, y_pred_labels, target_names=[f'Class {i}' for i in range(LABELS_LIMIT)])

print("Classification Report:")
print(report)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.45      0.65      0.53        20
     Class 1       0.67      0.70      0.68        20
     Class 2       0.72      0.65      0.68        20
     Class 3       0.65      0.65      0.65        20
     Class 4       0.62      0.75      0.68        20
     Class 5       0.82      0.70      0.76        20
     Class 6       0.67      0.50      0.57        20
     Class 7       0.75      0.75      0.75        20
     Class 8       0.72      0.65      0.68        20
     Class 9       0.55      0.60      0.57        20
    Class 10       0.38      0.40      0.39        20
    Class 11       0.94      0.85      0.89        20
    Class 12       0.67      0.70      0.68        20
    Class 13       0.65      0.75      0.70        20
    Class 14       0.58      0.55      0.56        20
    Class 15       0.76      0.80      0.78        20
    Class 16       